In [5]:
import pandas as pd
from typing import List

# FUNCTIONS
def get_sheet_names(file_path: str) -> List[str]:
    """
    Get the names of all sheets in an Excel file.

    Args:
    file_path (str): The path to the Excel file from which the sheet names are to be retrieved.

    Returns:
    List[str]: A list of strings, where each string is the name of a sheet in the Excel file.

    Functionality:
    - Reads an Excel file using pandas.
    - Retrieves the names of all sheets in the Excel file.
    """
    # Read the excel file using pandas
    xls = pd.ExcelFile(file_path)
    
    # Get the names of all the sheets in the excel file
    sheet_names = xls.sheet_names
    
    # Return the list of sheet names
    return sheet_names


# 1. Extract

In [6]:
# 1. Get sheet names from xlsx file
file_path = "data/ingreso_joyas_plata.xlsx"
sheet_names = get_sheet_names(file_path)
print("Sheet names:")
print(sheet_names)

Sheet names:
['anillos_plata', 'dijes_plata', 'juegos_plata', 'collares_plata', 'aretes_plata', 'cadena_plata', 'pulseras_plata', 'varios_plata', 'juegos_piedras_perlas', 'proveedores']


In [7]:
# 2. Use pop method to exclude last sheet (proveedores) from the data consolidation process
# Then read provedores sheet in adifferent df
proveedores_name = sheet_names.pop()
proveedores_df = pd.read_excel(file_path, sheet_name=proveedores_name)
proveedores_df.shape

(66, 3)

In [8]:
# 3. Use a for loop to read spreadsheets in sheet_names and append them in a list
# Replace the word "inventario" in column names with "ingreso" to simplify consolidation
list_of_df = []
for s in sheet_names:
    print(f"Sheet Name: {s}")
    # Read sheet and add new column with sheet name
    df = pd.read_excel(file_path, sheet_name=s)
    df["source"] = s

    # Replace "invetario" in column names with "ingreso"
    df.columns = [c.replace("inventario", "ingreso") for c in df.columns]
    
    # Append and print info
    list_of_df.append(df)
    print(f"Shape: {df.shape}")
    print(f"Column Names: {df.columns}\n")

Sheet Name: anillos_plata
Shape: (600, 15)
Column Names: Index(['fecha_compra', 'codigo', 'talla', 'ingreso_14/03/23',
       'ingreso_25/04/23', 'ingreso_27/03/24', 'ingreso_09/05/24', 'peso',
       'costo_gramo', 'costo', 'pvp', 'detalle', 'proveedor', 'nota',
       'source'],
      dtype='object')

Sheet Name: dijes_plata
Shape: (726, 20)
Column Names: Index(['fecha_compra', 'codigo', 'ingreso_14/03/23', 'ingreso_28/04/23',
       'ingreso_15/06/23', 'ingreso_18/08/23', 'ingreso_28/09/23',
       'ingreso_19/12/23', 'ingreso_07/03/24', 'ingreso_10/05/24',
       'ingreso_20/06/24', 'ingreso_08/07/24', 'peso', 'costo_gramo', 'costo',
       'pvp', 'detalle', 'proveedor', 'nota', 'source'],
      dtype='object')

Sheet Name: juegos_plata
Shape: (215, 18)
Column Names: Index(['fecha_compra', 'codigo', 'ingreso_14/03/23', 'ingreso_28/04/23',
       'ingreso_20/06/23', 'ingreso_28/09/23', 'ingreso_18/11/23',
       'ingreso_15/12/23', 'ingreso_07/03/24', 'ingreso_16/05/24', 'peso',
   

In [9]:
# Consolidate (concat) spreadsheets in one dataframe
all = pd.concat(list_of_df)
print(f"Sheet Name: all")
print(f"Shape: {all.shape}")
all.info()

Sheet Name: all
Shape: (2584, 37)
<class 'pandas.core.frame.DataFrame'>
Index: 2584 entries, 0 to 8
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fecha_compra      2584 non-null   datetime64[ns]
 1   codigo            2584 non-null   object        
 2   talla             599 non-null    object        
 3   ingreso_14/03/23  1784 non-null   float64       
 4   ingreso_25/04/23  74 non-null     float64       
 5   ingreso_27/03/24  1 non-null      float64       
 6   ingreso_09/05/24  51 non-null     float64       
 7   peso              2575 non-null   float64       
 8   costo_gramo       2575 non-null   float64       
 9   costo             2584 non-null   float64       
 10  pvp               2584 non-null   float64       
 11  detalle           2584 non-null   object        
 12  proveedor         2584 non-null   object        
 13  nota              448 non-null    object        
 14

In [10]:
# I noticed costo is of type object which means there are mixed types in it
# The following code catches elements of type str in the column so we can fix it in the file
costo_is_str_instance = all.costo.apply(lambda x: isinstance(x, str))
all[costo_is_str_instance][["codigo", "source"]]

,codigo,source


# 2. Transform (data quality)

- 2.1 Verify data types of every column (manual)
- 2.2 Check missing values for each column
- 2.3 Check ```peso``` or ```costo_gramo``` equal to ```0```
- 2.4 Rename provedores using ```proveedores_df```
- 2.5 Melt: Dataframe from wide to long format

In [11]:
# 2.2 Check missing values per columns
print(all.isna().sum())

# A. RESULTS
# COLNAME: FINDINGS / ACTIONS:
# fecha_compra: 4 missings/ Inspect and correct spreadsheet
# talla: only anillos should have non missings / No action needed
# ingreso_: missings are expected / No action needed
# peso: 9 missings / Inspect and corrected in file
# costo_gramo: 11 missings / Inspect and corrected in file

# B. NOTES:
# For juegos_piedras_perlas is ok to have missings in peso and costo gramo

fecha_compra           0
codigo                 0
talla               1985
ingreso_14/03/23     800
ingreso_25/04/23    2510
ingreso_27/03/24    2583
ingreso_09/05/24    2533
peso                   9
costo_gramo            9
costo                  0
pvp                    0
detalle                0
proveedor              0
nota                2136
source                 0
ingreso_28/04/23    2489
ingreso_15/06/23    2525
ingreso_18/08/23    2511
ingreso_28/09/23    2553
ingreso_19/12/23    2523
ingreso_07/03/24    2518
ingreso_10/05/24    2510
ingreso_20/06/24    2571
ingreso_08/07/24    2572
ingreso_20/06/23    2551
ingreso_18/11/23    2476
ingreso_15/12/23    2448
ingreso_16/05/24    2522
ingreso_26/04/23    2524
ingreso_05/06/24    2565
ingreso_21/04/23    2561
ingreso_14/06/24    2555
ingreso_26/03/24    2581
ingreso_23/02/23    2581
ingreso_08/03/24    2580
ingreso_29/03/23    2576
ingreso_04/06/24    2583
dtype: int64


In [12]:
# INSPECT:
# fecha_compra
print('Missings in fecha_compra:')
print(all.loc[all.fecha_compra.isnull(), ['codigo', 'source']])

# peso
print('\nMissings in peso:')
print(all.loc[all.peso.isnull(), ['codigo', 'source']])

# costo_gramo
print('\nMissings in costo_gramo:')
print(all.loc[all.costo_gramo.isnull(), ['codigo', 'source']])

# costo
print('\nMissings in costo:')
print(all.loc[all.costo.isnull(), ['codigo', 'source']])

# costo
print('\nMissings proveedor:')
print(all.loc[all.proveedor.isnull(), ['codigo', 'source']])

Missings in fecha_compra:
Empty DataFrame
Columns: [codigo, source]
Index: []

Missings in peso:
  codigo                 source
0  JV001  juegos_piedras_perlas
1  JV002  juegos_piedras_perlas
2  JV003  juegos_piedras_perlas
3  JV004  juegos_piedras_perlas
4  JV005  juegos_piedras_perlas
5  JV006  juegos_piedras_perlas
6  JV007  juegos_piedras_perlas
7  JV008  juegos_piedras_perlas
8  JV009  juegos_piedras_perlas

Missings in costo_gramo:
  codigo                 source
0  JV001  juegos_piedras_perlas
1  JV002  juegos_piedras_perlas
2  JV003  juegos_piedras_perlas
3  JV004  juegos_piedras_perlas
4  JV005  juegos_piedras_perlas
5  JV006  juegos_piedras_perlas
6  JV007  juegos_piedras_perlas
7  JV008  juegos_piedras_perlas
8  JV009  juegos_piedras_perlas

Missings in costo:
Empty DataFrame
Columns: [codigo, source]
Index: []

Missings proveedor:
Empty DataFrame
Columns: [codigo, source]
Index: []


In [13]:
# 2.3 Check peso or costo_gramo <= 0
# ACTION NEEDED: Manually fix the file with non-zero values
print('Rows with costo_gramo <= 0')
print(all[all.costo_gramo<=0][['codigo', 'source', 'costo_gramo', 'peso', 'costo']])

print('\nRows with peso <= 0')
print(all[all.peso<=0][['codigo', 'source', 'costo_gramo', 'peso', 'costo']])

print('\nRows with costo <= 0 as consequense of peso <= 0 OR costo_gramo <= 0')
print(all[all.costo<=0][['codigo', 'source', 'costo_gramo', 'peso', 'costo']])

Rows with costo_gramo <= 0
Empty DataFrame
Columns: [codigo, source, costo_gramo, peso, costo]
Index: []

Rows with peso <= 0
Empty DataFrame
Columns: [codigo, source, costo_gramo, peso, costo]
Index: []

Rows with costo <= 0 as consequense of peso <= 0 OR costo_gramo <= 0
Empty DataFrame
Columns: [codigo, source, costo_gramo, peso, costo]
Index: []


In [14]:
# 2.4 Rename proveedores
# Create dictionary where key is old name and values are new name
dict_of_proveedores_1 = {k:v for k,v in zip(proveedores_df.Proveedor, proveedores_df['Nuevo Nombre'])}

# Use the dictionary to replace old values
all['proveedor'] = all.proveedor.replace(dict_of_proveedores_1)

# Validate unuique values. Create a list of values that need manual changes
print(f'Unique providers after 1st iteration: \n{all.proveedor.unique()} \n')

# Create new dictionary for remaining values {old_name : new_name}
dict_of_proveedores_2 = {
    'China':'CHINA', 
    'Cirkon ':'CIRKON', 
    'Cirkon  ':'CIRKON',
    'flavio jara':'FLAVIO JARA', 
    'CAMBIO DE CODIGO':'VOGA',
        ' ':'VOGA',
        '*': 'VOGA',
        'LX, N.Y.':'LX USA',
        'Alina ' :'ALINA PAZ',
        'Andres ' :'ANDRES CADAVID',
        'Feria Mia' : 'MIAMI',
        'pedir': 'VOGA'
    }

# Use the new dictionary to replace remaining values
all['proveedor'] = all.proveedor.replace(dict_of_proveedores_2)

# Validate unique names again
print(f'Unique providers after 2nd iteration: \n{all.proveedor.unique()}')


Unique providers after 1st iteration: 
['VOGA' 'SOLEDAD SAENZ' 'China' 'ALPHA TRADING' 'XAVIER GUILLEN'
 'ALEX CASTRO' 'RIOBAMBA' 'ANDRES CADAVID' 'CIRKON' 'flavio jara'
 'JUAN CHALCO' 'ALINA PAZ' 'PALACIO JOYAS' 'SEMPERTEGUI' 'MILANUS USA'
 'LX USA' 'FLAVIO JARA' 'CHINA' 'Cirkon  ' 'ZAFIRO CHORDELEG'
 'CAMBIO DE CODIGO' 'USA' ' ' 'P&K' 'BODY JEWELZ' '*' 'LX, N.Y.' 'Alina '
 'Andres ' 'Feria Mia' 'pedir' 'FERNANDO JARA' 'PANAMA' 'Cirkon '
 'GUILLERMO PARRA'] 

Unique providers after 2nd iteration: 
['VOGA' 'SOLEDAD SAENZ' 'CHINA' 'ALPHA TRADING' 'XAVIER GUILLEN'
 'ALEX CASTRO' 'RIOBAMBA' 'ANDRES CADAVID' 'CIRKON' 'FLAVIO JARA'
 'JUAN CHALCO' 'ALINA PAZ' 'PALACIO JOYAS' 'SEMPERTEGUI' 'MILANUS USA'
 'LX USA' 'ZAFIRO CHORDELEG' 'USA' 'P&K' 'BODY JEWELZ' 'MIAMI'
 'FERNANDO JARA' 'PANAMA' 'GUILLERMO PARRA']


## 2.5 Melt

The pd.melt function in pandas is used to transform a DataFrame from a wide format to a long format. In the wide format, data is typically spread across multiple columns, while in the long format, data is stacked in a single column with an additional column indicating the original variable name (typically the column name in the wide format).

Parameters of pd.melt:

	• id_vars: Specifies the columns to keep unchanged (identifier variables). These columns remain unpivoted.
	• value_vars (optional): Specifies the columns to unpivot. If not provided, all columns not specified in id_vars are used.
	• var_name: The name to use for the ‘variable’ column in the resulting DataFrame.
	• value_name: The name to use for the ‘value’ column in the resulting DataFrame.

In [15]:
# Especificar las columnas que no deben ser melted
id_vars = ['fecha_compra', 'codigo', 'talla', 'peso',
       'costo_gramo', 'costo', 'pvp', 'detalle', 'proveedor', 'nota', 'source']

# Realizar el melt del dataframe
all_melted = pd.melt(all, id_vars=id_vars, var_name='fecha_ingreso', value_name='count_items')

# Limpiar la columna 'fecha' para extraer la fecha en el formato correcto
all_melted['fecha_ingreso'] = all_melted['fecha_ingreso'].str.replace('ingreso_', '')

# Eliminar las filas con valores perdidos en la columna 'items'
# Esto corresponde a items que no tienen ingresos registrados
all_melted.dropna(subset=['count_items'], inplace=True)

all_melted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2883 entries, 0 to 67183
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   fecha_compra   2883 non-null   datetime64[ns]
 1   codigo         2883 non-null   object        
 2   talla          582 non-null    object        
 3   peso           2874 non-null   float64       
 4   costo_gramo    2874 non-null   float64       
 5   costo          2883 non-null   float64       
 6   pvp            2883 non-null   float64       
 7   detalle        2883 non-null   object        
 8   proveedor      2883 non-null   object        
 9   nota           610 non-null    object        
 10  source         2883 non-null   object        
 11  fecha_ingreso  2883 non-null   object        
 12  count_items    2883 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(7)
memory usage: 315.3+ KB


In [16]:
# Catch values with count_items == 0 and clean in file
print("Count items with values = 0")
print(all_melted[all_melted.count_items==0][['codigo', 'source', 'fecha_ingreso', 'count_items']])

print("\n Lenght of fecha_ingreso")
print(all_melted.fecha_ingreso.str.len().value_counts(dropna=False))

Count items with values = 0
Empty DataFrame
Columns: [codigo, source, fecha_ingreso, count_items]
Index: []

 Lenght of fecha_ingreso
fecha_ingreso
8    2883
Name: count, dtype: int64


In [17]:
# Count items to int64
all_melted["count_items"] = all_melted.count_items.astype("Int64")

# fecha_ingreso to datetime- Format "dd/mm/yy"
all_melted["fecha_ingreso"] = pd.to_datetime(all_melted.fecha_ingreso, format="%d/%m/%y", errors='raise')  

all_melted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2883 entries, 0 to 67183
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   fecha_compra   2883 non-null   datetime64[ns]
 1   codigo         2883 non-null   object        
 2   talla          582 non-null    object        
 3   peso           2874 non-null   float64       
 4   costo_gramo    2874 non-null   float64       
 5   costo          2883 non-null   float64       
 6   pvp            2883 non-null   float64       
 7   detalle        2883 non-null   object        
 8   proveedor      2883 non-null   object        
 9   nota           610 non-null    object        
 10  source         2883 non-null   object        
 11  fecha_ingreso  2883 non-null   datetime64[ns]
 12  count_items    2883 non-null   Int64         
dtypes: Int64(1), datetime64[ns](2), float64(4), object(6)
memory usage: 318.1+ KB


In [18]:
all_melted.head()

,fecha_compra,codigo,talla,peso,costo_gramo,costo,pvp,detalle,proveedor,nota,source,fecha_ingreso,count_items
0,2021-11-22,AP001,T6,2.3,2.4,5.52,18.7,llano 2 máscaras,VOGA,NaN,anillos_plata,2023-03-14,2
1,2021-11-22,AP001,T7,2.3,2.4,5.52,18.7,llano 2 máscaras,VOGA,NaN,anillos_plata,2023-03-14,1
2,2021-11-22,AP001,T9,2.3,2.4,5.52,18.7,llano 2 máscaras,VOGA,NaN,anillos_plata,2023-03-14,1
3,2021-11-22,AP002,T4,1.4,2.4,3.36,17.8,llano circulo figura grabada,VOGA,NaN,anillos_plata,2023-03-14,2
4,2021-11-22,AP002,T5,1.4,2.4,3.36,17.8,llano circulo figura grabada,VOGA,NaN,anillos_plata,2023-03-14,1


# 3. Load

In [20]:
# 4. Save dataframe as Excel
all_melted.to_excel("data/clean_files/all_plata_melted.xlsx", index=False)